### SparkNLP Word Analaysis POS Lemma

In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.ml import PipelineModel
import sparknlp
import sys
import pandas as pd
import numpy as np
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.pretrained import ResourceDownloader
from pyspark.sql.types import StringType, IntegerType
from pathlib import Path
if sys.version_info[0] < 3:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve

In [ ]:
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

#### POS

In [ ]:
data = spark.read.text('text_all_word.txt').toDF('text')